----
# ANÁLISE ESTATÍSTICA DOS ACIDENTES NAS RODOVIAS FEDERAIS BRASILEIRAS 

* Leandro Alencar – 1931133007
* Maycon Alves – 1931133015
* Nilson Michiles - 1931133032

## Table of contents
* [Préprocessamento de Dados](#1)
* [Análises Descritivas e Estatísticas](#2)
* [Análises de Componentes Principais](#3)
* [K-Means (K-médias)](#4)
* [Série Temporal](#5)
----

# Préprocessamento de Dados <a name="1"></a>
Como etapa inicial, será necessário realizar o préprocessamento dos dados a fim de remover eventuais registros duplicados, remover ou substituir registros nulos e aplicar a correta formatação dos dados (data, númerico, categórico, etc).

> Foram encontrados registros nulos com o campo escrito "(null)", foi necessário desenvolver função customizada para, também, atender a essas peculiaridades.

A função em seguida tem por objetivo:
- remover registros duplicados;
- substituir valores nulos numéricos por -1 (e também valores de string como "(null)" em colunas numéricas que foram encontrados);
- valores nulos qualitativos pelo registro mais frequente, dada que a quantidade encontrada foi mínima (máximo de 30 registros "(null)" por coluna)
- manter e ordenar as colunas que aparecem em todos os datasets - os anos de 2017 a 2019 possuem as colunas "latitude", "longitude", "regional", "delegacia" e "uop", que não constam nos datasets anteriores.

In [ ]:
# Bibliotecas
import os
import pandas as pd
import re
import unidecode
import numpy as np
import warnings
import seaborn as sns
sns.set()
sns.set_context("paper")
warnings.filterwarnings("ignore")

In [ ]:
def cleanDF(df) -> pd.DataFrame:
    # Remover Duplicados
    df = df.drop_duplicates(keep='first')
    
    # Transformar coluna KM e BR, em que há strings, em float e substituir "(null)" por -1
    if df['km'].dtype == 'object':
        df['km'] = df['km'].str.replace(r'\(null\)', '-1.0')
        df['km'] = df['km'].str.replace(',', '.').apply(float)
        
    if df['br'].dtype == 'object':
        df['br'] = df['br'].str.replace(r'\(null\)', '-1.0')
        df['br'] = df['br'].str.replace(',', '.').apply(float)
        
    else:
        pass

    # Formatar para minusculo, remover acentos e espacos
    for col in df.select_dtypes(include='object').columns:
        if df[col].isna().sum() > 0:
            df[col] = df[col].replace(np.nan, df[col].value_counts().idxmax())
        else:
            pass
        try:
            df[col] = df[col].apply(lambda x: unidecode.unidecode(x).lower().strip())
        except:
            print('Error in col: ', col)
    
    # Formatar Data YYYY-MM-DD
    df['data_inversa'] = pd.to_datetime(df['data_inversa'], dayfirst=True)
    df['ano'] = df['data_inversa'].dt.year
    df['mes'] =  df['data_inversa'].dt.month
    
    
    # Ordenação das colunas
    cols = ['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km',
       'municipio', 'causa_acidente', 'tipo_acidente',
       'classificacao_acidente', 'fase_dia', 'sentido_via',
       'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo',
       'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos',
       'ignorados', 'feridos', 'veiculos','ano','mes']
    
    df = df[cols]
    return df

### Importação, tratamento dos dados e concatenação

In [ ]:
%%time
for dirname, _, filenames in os.walk('/kaggle/input'):
    dfs = list()
    for filename in filenames:
        try:
            df = pd.read_csv(os.path.join(dirname, filename), sep=';', encoding='latin1', low_memory=False)
            dfs.append(cleanDF(df))
        except:
            print("Error in file: ", filename)
      
acidentes_df = pd.concat(dfs, ignore_index=True )

### Exploração quanto a ocorrências de "Missings"

In [ ]:
acidentes_df.info()

### Tratamento dos missings

In [ ]:
%%time
acidentes_df = acidentes_df.replace(np.nan, -1)

obj_nulls = list()
for col in acidentes_df.select_dtypes(include='object').columns:
    if acidentes_df[col].str.contains('null').sum() > 0:
        obj_nulls.append(col)
        print(col)
    else:
        pass

Onde a UF era missing, foi utilizado o valor em que o municipio era o mesmo e a UF não era nulo para a correção:

In [ ]:
%%time
for mun in acidentes_df[acidentes_df[obj_nulls[0]].str.contains('null')]['municipio']:
    acidentes_df[obj_nulls[0]][acidentes_df['municipio'] == mun] = acidentes_df[acidentes_df['municipio'] == mun][obj_nulls[0]].mode()[0]

In [ ]:
acidentes_df[acidentes_df['causa_acidente'] == '(null)']['causa_acidente'] = 'outras'

Nos demais, foram substituidos pelo valor mais frequente.

In [ ]:
%%time
for col in obj_nulls[1:]:
    acidentes_df[col][acidentes_df[col].str.contains('null')] = acidentes_df[col].mode()[0]

### Dataframe sem missings

In [ ]:
acidentes_df.isnull().sum()

In [ ]:
acidentes_df = acidentes_df.sort_values(by='data_inversa')
acidentes_df.head(1).T

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acidentes_df.hist(figsize=(20,10))

### Feature Engineering
Criação de uma coluna chamada "regiao" para possibilitar análises de dados agrupados.

In [ ]:
def regiao(x):
    
    if x in ['al' , 'ba', 'ce', 'ma', 'pb', 'pe', 'pi', 'rn','se']:
        return 'nordeste'
    if x in ['ac' , 'ap', 'am', 'pa', 'ro', 'rr', 'to']:
        return 'norte'
    if x in ['df' , 'go', 'ms', 'mt']:
        return 'centro oeste'
    if x in ['es' , 'mg', 'sp', 'rj']:
        return 'sudeste'
    if x in ['pr' , 'sc', 'rs']:
        return 'sul'
    


acidentes_df['regiao'] = acidentes_df['uf'].transform(regiao)

# Removendo a causa de acidente "outras", pois não informa muita coisa.
acidentes_df = acidentes_df[acidentes_df['causa_acidente'] != 'outras']

In [ ]:
#gerando output do DF para gerar a MCA - Analise de correspondencia multipla no R
acidentes_df.to_csv('acidentes_df.csv')

# Análise Descritiva Exploratória <a name="2"></a>

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Caso nao queira rodar toda a etapa de tratamento dos dados
#acidentes_df = pd.read_csv('./kaggle/acidentes_df_csv')

In [ ]:
#import pandas_profiling as pp
#pp.ProfileReport(acidentes_df)

# Análise Estatística

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

ax = sns.countplot(x="regiao", hue="causa_acidente", data=acidentes_df[acidentes_df['regiao'] == 'norte'], palette="Set3")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax = sns.countplot(x="regiao", hue="causa_acidente", data=acidentes_df[acidentes_df['regiao'] == 'nordeste'], palette="Set3")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax = sns.countplot(x="regiao", hue="causa_acidente", data=acidentes_df[acidentes_df['regiao'] == 'centro oeste'], palette="Set3")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax = sns.countplot(x="regiao", hue="causa_acidente", data=acidentes_df[acidentes_df['regiao'] == 'sul'], palette="Set3")

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
ax = sns.countplot(x="regiao", hue="causa_acidente", data=acidentes_df[acidentes_df['regiao'] == 'sudeste'], palette="Set3")

# Análise de Componentes Principais <a name="3"></a>

A Análise de Componentes Principais (em inglês PCA) é o nome comum dado à técnica que usa princípios de álgebra linear para transformar variáveis, possivelmente correlacionadas, em um número menor de variáveis chamadas de Componentes Principais.

Iremos analisar as causas mais comuns de acidente por região no Brasil.

## Causa de acidentes x Região

### Criando tabela cruzada entre região e causas de acidentes.

In [ ]:
centro_oeste = acidentes_df[acidentes_df['regiao'] == 'centro oeste']

In [ ]:
centro_oeste = centro_oeste[['causa_acidente', 'municipio']]
causa_acidentes_x_municipio = pd.crosstab(centro_oeste['causa_acidente'], centro_oeste['municipio'])

In [ ]:
print(causa_acidentes_x_municipio.shape)
causa_acidentes_x_municipio

Sem realizar ajustes lineares para cada par de dados fica quase impossível visualizar algum padrão ou tendência nos dados acima. Esse é um caso onda a PCA pode ajudar. 

Definindo uma função para normalizar os dados.

In [ ]:
def z_score(x):
    """Remove a média e normaliza os pelo desvio padrão"""
    return (x - x.mean()) / x.std()

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=None)
score = pca.fit_transform(causa_acidentes_x_municipio.apply(z_score))

In [ ]:
loadings = pd.DataFrame(pca.components_)
loadings.index =   ['PC %s' % pc for pc in loadings.index + 1]
loadings.columns = causa_acidentes_x_municipio.columns
loadings

In [ ]:
PCs = np.dot(loadings.values.T, causa_acidentes_x_municipio)

In [ ]:
font = {'family' : 'monospace',
        'weight' : 'normal',
        'size'   : 14}

plt.rc('font', **font)

 ### Plotando a primeira dimensão(PC1)

In [ ]:
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)

fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(PCs[0], np.zeros_like(PCs[0]), label="Scores", **marker)

[ax.text(x, y, t) for x, y, t in zip(PCs[0], loadings.values[0, :], loadings.columns)]

ax.set_xlabel("PC1")

_ = ax.set_ylim(-1, 1)
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)


Facilmente vemos que a região sul e sudeste têm uma causa de acidente diferente do norte, centro oeste e nordeste, esses três últimos parecem se agrupar em um grupo de causa de acidente similar.

 ### Plotando a primeira e segunda dimensão(PC1 e PC2)

Agora vamos plotar a primeira e segunda PCs juntas. Esse tipo de gráfico é chamado de Score plot.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(PCs[0], PCs[1], label="Scores", **marker)

ax.set_xlabel("PC1")
ax.set_ylabel("PC2")

text = [ax.text(x, y, t) for x, y, t in zip(PCs[0], PCs[1], loadings.columns)]


Note que na "segunda" dimensão estão as diferenças entre as três regiões que agrupamos no gráfico anterior.

In [ ]:
perc = pca.explained_variance_ratio_ * 100

perc = pd.DataFrame(perc, columns=['Percentual de razão explicada'], index=['PC %s' % pc for pc in np.arange(len(perc)) + 1])
ax = perc.plot(kind='bar')

Em geral se busca componentes o suficiente para explicar entre 70-80% dos dados. nota-se que utlizamos mais que 70% da variância dos dados. Saímos de uma dimensão de (28 x 228) para uma dimensão de (2x5).

In [ ]:
marker = dict(linestyle='none', marker='o', markersize=7, color='blue', alpha=0.5)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(loadings.iloc[:, 0], loadings.iloc[:, 1], label="Loadings", **marker)
ax.set_xlabel("non-projected PC1")
ax.set_ylabel("non-projected PC2")
ax.axis([-1, 1, -1, 1])
text = [ax.text(x, y, t) for x, y, t in zip(loadings.iloc[:, 0], loadings.iloc[:, 1], causa_acidentes_x_municipio.index)]

Outro gráfico comum para explorar os resultados é o Loadings plot ou seja, a influência de cada variável original nas componentes principais. Note que velocidade incompatível, defeito mecânico em veículo, desobediência de sinalização, não guardar distância de segurança e falta de atenção a condução se destacam da aglomeração central.

# K-Means <a name="4"></a>

In [ ]:
def describe_cluster(variavel, cluster_id):
    
    for x in range(0, cluster_id):
        
        
        font = {'family' : 'monospace',
        'weight' : 'normal',
        'size'   : 22}

        plt.rc('font', **font)
    
        fig, ax = plt.subplots(figsize=(26, 10))

        plt.subplot(1, 2, 1)

        summary = pd.DataFrame(variavel[variavel['cluster_id'] == x].describe()).T
        summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
        summary = summary.T

        table = plt.table(cellText=summary.values,
                  rowLabels=summary.index,
                  colLabels=summary.columns,
                              cellLoc = 'right', rowLoc = 'center',
          loc='right', bbox=[.1,.05,1.3,.95])

        plt.axis('off')

        plt.title("Descrição do Cluster " + str(x) + " - Quantidade de Acidentes ")
        table.set_fontsize(22)
        table.scale(3, 3)  
        
        plt.subplot(1, 2, 2)
        plt.title("Amostra dos Municípios do Cluster " + str(x))
        quantidade = variavel[variavel['cluster_id'] == x]['cluster_id'].count()
        if  quantidade < 10:
            sample = variavel[variavel['cluster_id'] == x].sample(quantidade).index
        else: 
            sample = variavel[variavel['cluster_id'] == x].sample(10).index
            
        table = plt.table(cellText=pd.DataFrame(sample).values,
                          cellLoc = 'right', rowLoc = 'center',
          loc='top',
                          bbox=[.25,.55,.45,.45])

        plt.axis('off')
        # may help

In [ ]:
# Importar o algoritimo/modelo
from sklearn.cluster import KMeans

In [ ]:
causa_acidentes_x_regiao_t = causa_acidentes_x_municipio.T

#### K-Means - Falta de Atenção x Ingestão de Alcool

In [ ]:
W = causa_acidentes_x_regiao_t[['falta de atencao', 'ingestao de alcool']]

fig, ax = plt.subplots(figsize=(16, 5))


summary = pd.DataFrame(W.describe()).T
summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
summary = summary.T

table = plt.table(cellText=summary.values,
          rowLabels=summary.index,
          colLabels=summary.columns,
          cellLoc = 'right', rowLoc = 'center',
          loc='right', bbox=[.1,.05,.75,.75])

plt.axis('off')

table.set_fontsize(22)
table.scale(3, 3)  # may help

In [ ]:
# Método Elbow
# Cálculo do SSE - Sum of Squared Erros
sse = []

for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(W)
    sse.append(kmeans.inertia_)

In [ ]:
# Plotando o gráfico
import matplotlib.pyplot as plt

plt.plot(range(1, 15), sse, 'bx-')
plt.title('Método Elbow')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.xticks(range(1, 15))
plt.show()

In [ ]:
# Vamos usar 3 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_id = kmeans.fit_predict(W)

In [ ]:
# Agora vamos guardar os resultados no dataframe
W['cluster_id'] = cluster_id

In [ ]:
describe_cluster(W, 4)

In [ ]:
# Plotando os agrupamentos e os centroídes
fig, ax = plt.subplots(figsize=(10, 5))



sns.scatterplot(x="falta de atencao", y="ingestao de alcool", hue="cluster_id", data=W, s=200, palette="viridis")
plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text



ax.scatter(kmeans.cluster_centers_[:,0] ,
           kmeans.cluster_centers_[:,1], 
           color='red', 
           marker="x", s=100)


#text = [ax.text(x, y, t) for x, y, t in zip(W.values[:,0], 
#                                            W.values[:,1], 
#                                            W.values[:,2])]



plt.xlabel('Falta de Atenção')
plt.ylabel('Ingestão de Alcool')
plt.show()

#### K-Means - Falta de Atenção x Velocidade Incompatível

In [ ]:
X = causa_acidentes_x_regiao_t[['falta de atencao', 'velocidade incompativel']]
fig, ax = plt.subplots(figsize=(16, 5))


summary = pd.DataFrame(X.describe()).T
summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
summary = summary.T

table = plt.table(cellText=summary.values,
          rowLabels=summary.index,
          colLabels=summary.columns,
          cellLoc = 'right', rowLoc = 'center',
          loc='right', bbox=[.1,.05,.75,.75])

plt.axis('off')

table.set_fontsize(22)
table.scale(3, 3)  # may help

In [ ]:
# Método Elbow
# Cálculo do SSE - Sum of Squared Erros
sse = []

for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
    sse.append(kmeans.inertia_)

In [ ]:
# Plotando o gráfico
import matplotlib.pyplot as plt

plt.plot(range(1, 15), sse, 'bx-')
plt.title('Método Elbow')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.xticks(range(1, 15))
plt.show()

In [ ]:
# Vamos usar 3 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_id = kmeans.fit_predict(X)

In [ ]:
# Agora vamos guardar os resultados no dataframe
X['cluster_id'] = cluster_id



In [ ]:
describe_cluster(X, 4)

In [ ]:
#Plotando os agrupamentos e os centroídes
fig, ax = plt.subplots(figsize=(10, 5))



sns.scatterplot(x="falta de atencao", y="velocidade incompativel", hue="cluster_id", data=X, s=300, palette="viridis")
plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text


ax.scatter(kmeans.cluster_centers_[:,0] ,
           kmeans.cluster_centers_[:,1], 
           color='red', 
           marker="x", s=100)



plt.xlabel('Falta de Atenção')
plt.ylabel('Velocidade Incompatível')

plt.show()

#### K-Means - Não guardar distância de segurança x Velocidade incompatível

In [ ]:
Y = causa_acidentes_x_regiao_t[['nao guardar distancia de seguranca', 'velocidade incompativel']]

fig, ax = plt.subplots(figsize=(16, 5))


summary = pd.DataFrame(Y.describe()).T
summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
summary = summary.T

table = plt.table(cellText=summary.values,
          rowLabels=summary.index,
          colLabels=summary.columns,
          cellLoc = 'right', rowLoc = 'center',
          loc='right', bbox=[.1,.05,.75,.75])

plt.axis('off')

table.set_fontsize(22)
table.scale(3, 3)  # may help

In [ ]:
# Método Elbow
# Cálculo do SSE - Sum of Squared Erros
sse = []

for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(Y)
    sse.append(kmeans.inertia_)

In [ ]:
# Plotando o gráfico
import matplotlib.pyplot as plt

plt.plot(range(1, 15), sse, 'bx-')
plt.title('Método Elbow')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.xticks(range(1, 15))
plt.show()

In [ ]:
# Vamos usar 3 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_id = kmeans.fit_predict(Y)

In [ ]:
# Agora vamos guardar os resultados no dataframe
Y['cluster_id'] = cluster_id


In [ ]:
describe_cluster(Y, 4)

In [ ]:
#Plotando os agrupamentos e os centroídes
fig, ax = plt.subplots(figsize=(10, 5))



sns.scatterplot(x="nao guardar distancia de seguranca", y="velocidade incompativel", hue="cluster_id", data=Y, s=300, palette="viridis")
plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text


ax.scatter(kmeans.cluster_centers_[:,0] ,
           kmeans.cluster_centers_[:,1], 
           color='red', 
           marker="x", s=100)



plt.xlabel('Não guardar distância de segurança')
plt.ylabel('Velocidade incompatível')
plt.show()

Levando em considereção as causas de acidentes: Não guardar distância de segurança. Velocidade Incompatível, Falta de Atenção e Ingestão de Alcool.

Facilmente vemos que Norte e Centro-Oeste têm uma quantidade de acidentes bem menor que Nordeste, Sudeste e Sul, dessa forma se agrupam em um cluster que tem como caracteristica o menor numero de acidentes por essas causas.

Sul e Sudeste se agrupam pois tem valores altos de acidentes por essas causas, sendo que nessa região é onde se possuem mais rodovias pavimentadas.

Nordeste fica em um grupo sozinho. tem quantidade de acidentes por essas causas abaixo de sul e sudeste e acima de norte e centro-oeste.

#### K-Means - PC 1 x PC 2

In [ ]:
pcs = pd.DataFrame(PCs)

pcs.columns = loadings.columns

Z = pcs.T[[0, 1]]

fig, ax = plt.subplots(figsize=(16, 5))


summary = pd.DataFrame(Z.describe()).T
summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
summary = summary.T

table = plt.table(cellText=summary.values,
          rowLabels=summary.index,
          colLabels=summary.columns,
          cellLoc = 'right', rowLoc = 'center',
          loc='right', bbox=[.1,.05,.75,.75])

plt.axis('off')

table.set_fontsize(22)
table.scale(3, 3)  # may help

In [ ]:
# Método Elbow
# Cálculo do SSE - Sum of Squared Erros
sse = []

for k in range(1, 15):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(Z)
    sse.append(kmeans.inertia_)

In [ ]:
# Plotando o gráfico
import matplotlib.pyplot as plt

plt.plot(range(1, 15), sse, 'bx-')
plt.title('Método Elbow')
plt.xlabel('Número de clusters')
plt.ylabel('SSE')
plt.xticks(range(1, 15))
plt.show()

In [ ]:
# Vamos usar 3 clusters
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_id = kmeans.fit_predict(Z)

In [ ]:
# Agora vamos guardar os resultados no dataframe
Z['cluster_id'] = cluster_id

In [ ]:
describe_cluster(Z, 4)

In [ ]:
#Plotando os agrupamentos e os centroídes
fig, ax = plt.subplots(figsize=(10, 5))



sns.scatterplot(x=0, y=1, hue="cluster_id", data=Z, s=300, palette="viridis")
plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text


ax.scatter(kmeans.cluster_centers_[:,0] ,
           kmeans.cluster_centers_[:,1], 
           color='red', 
           marker="x", s=100)




plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()


Realizando um agrupamento por componentes principais, vemos que Nordeste, Norte e Centro-Oeste já ficam no mesmo grupo, Sul e Sudeste ficam agora em grupos diferentes. como os componentes principais leva em consideração todas as causas de acidentes, trás uma maior compreensão da similaridade de acidentes entre as regiões.

# Série Temporal <a name="5"></a>

In [ ]:
# Bilbiotecas
!pip install pmdarima
import statsmodels.api as sm
from pmdarima.arima import auto_arima
import datetime as dt

Iremos realizar a previsão da quantidade de acidentes com vítimas para o ano de 2020 por meio de um modelo auto-regressivo integrado de médias móveis sazonal, em inglês chamado de SARIMA (Seasonal Autoregressive Integrated Moving Average). Séries Temporais, em geral, podem ser classificadas como aditivas ou multiplicativas mas ambas possuem os mesmos 3 componentes:

Tendência, que representa a direção geral como os dados se desenvolvem ao longo do tempo;
Sazonalidade, padrões de como os dados mudam em relação a período determinado;
Erro, são variações irregulares não explicadas pela tendência ou sazonalidade.
No caso de uma Séria Temporal Aditiva, um modelo pode ser explicado pela fórmula:

Y[t] = T[t] + S[t] + e[t]

> Onde:
- Y[t] : Saída do Modelo
- T[t] : Componente de Tendência do modelo
- S[t] : Componente de Sazonalidade do modelo
- e[t] : erro ou resíduo

Antes, façamos uma breve análise exploratória dos dados de acidentes totais e acidentes com vítimas.

In [ ]:
# Acidentes Time Series Centro Oeste
acidentes_ts = acidentes_df[(acidentes_df['regiao'] == 'centro oeste')]
acidentes_ts['com_vitimas'] = np.where((acidentes_ts['mortos'] > 0) |
                                       (acidentes_ts['feridos'] > 0) |
                                       (acidentes_ts['feridos_leves'] > 0) |
                                       (acidentes_ts['feridos_graves'] > 0),
                                       1,0)

acidentes_ts['mes'] = acidentes_ts['data_inversa'].dt.month.apply(lambda x: "{:02d}".format(x))
acidentes_ts['ano'] = acidentes_ts['data_inversa'].dt.year.astype(str)

# Acidentes com Vitimas
acidentes_cvit = acidentes_ts.groupby(['mes','ano'])['com_vitimas'].sum().reset_index()
acidentes_cvit['dia'] = '01'
acidentes_cvit['data'] = pd.to_datetime(acidentes_cvit['ano']+acidentes_cvit['mes']+acidentes_cvit['dia'])
acidentes_cvit = acidentes_cvit[['data','com_vitimas']].set_index('data').sort_values('data')

# Acidentes Totais
acidentes_tot = acidentes_ts.groupby(['mes','ano'])['com_vitimas'].count().reset_index()
acidentes_tot['dia'] = '01'
acidentes_tot['data'] = pd.to_datetime(acidentes_tot['ano']+acidentes_tot['mes']+acidentes_tot['dia'])
acidentes_tot = acidentes_tot[['data','com_vitimas']].set_index('data').rename(columns={'com_vitimas':'total'}).sort_values('data')

In [ ]:
ax = acidentes_tot.plot(title='Série Histórica', figsize=(20,10))
acidentes_cvit.plot(ax=ax)

plt.title('Acidentes nas rodovias do Centro-Oeste | 2009-2019', fontsize=15, rotation=0)
plt.xticks(fontsize=15, rotation=0)
plt.yticks(fontsize=15)
plt.xlabel('')

ax.legend(['Total de Acidentes', 'Acidentes com Vítimas'], fontsize=14)

plt.show()

Pode-se perceber que houve redução no número total de acidentes com a evolução do tempo, porém o número de acidentes com vítimas não acompanhou essa redução, aumentando do índice de acidentes com vítimas. 

No gráfico abaixo, fica mais evidente o aumento progressivo no número total de acidentes com vítimas. Houve redução no período de 2013 a 2016, porém um grande aumento em 2017 e tendência de crescimento  futuro, após sua leve queda em 2018.

In [ ]:
contagem = acidentes_ts.groupby('ano')['com_vitimas'].sum().reset_index().set_index('ano')

fig, ax = plt.subplots(1,1, figsize=(20,10))

plt.scatter(x=contagem.index, y=contagem['com_vitimas'], color='purple', linewidths=5, zorder=2)
plt.plot(contagem['com_vitimas'], linewidth=3, zorder=1)
plt.xticks(fontsize=18)
plt.tick_params(axis='y', which='both', left=False, labelleft=False)
plt.title('Total de Acidentes com Vítimas por Ano', size=20)

for data, count in zip(contagem.index, contagem.values): 
    ax.annotate(count[0]
               ,xytext=(data, count+50)
               ,fontsize=15 
               ,xy=(data, count)
               )
plt.show()

# Mortalidade nos Acidentes
A mortalidade dos acidentes nas rodovias do Centro-Oeste mostrou-se baixa em comparação com o total de acidentes com vítimas, e demonstra um comportamento, aparentemente, estável.

In [ ]:
# Mortos
acidentes_mort = acidentes_ts.groupby(['mes','ano'])['mortos'].sum().reset_index()
acidentes_mort['dia'] = '01'
acidentes_mort['data'] = pd.to_datetime(acidentes_mort['ano']+acidentes_mort['mes']+acidentes_mort['dia'])
acidentes_mort = acidentes_mort[['data','mortos']].set_index('data').sort_values('data')

ax = acidentes_cvit.plot(title='Série Histórica', figsize=(20,10))
acidentes_mort.plot(ax=ax)

plt.title('Mortalidade nos Acidentes em rodovias do Centro-Oeste | 2009-2019', fontsize=15, rotation=0)
plt.xticks(fontsize=15, rotation=0)
plt.yticks(fontsize=15)
plt.xlabel('')

ax.legend(['Acidentes com Vítimas', 'Acidentes com Óbitos'], fontsize=14)

plt.show()

## Decomposição da Série Temporal

É possível, e muito importante, analisarmos esses três componentes separadamente de modo a avaliar melhor o comportamento individual de cada item, para isso iremos realizar sua decomposição. Como nossos dados tratam-se de observações mensais, a frequência passada como parâmetro será 12, o que significa que para cada ponto iremos analisar a média dos 6 meses anteriores e 6 posteriores. Por meio da decomposição, é possível identificar que nossos dados possuem uma clara tendência de crescimento, e quanto à sazonalidade, o gráfico demonstra que existe efeito sazonal com picos máximos e mínimos nos meses de Dezembro e Fevereiro, respectivamente.

In [ ]:
res = sm.tsa.seasonal_decompose(acidentes_cvit,freq=12)
def plot_decompose(res):
    fig, axes = plt.subplots(ncols=1, nrows=4, sharex=True, figsize=(16,12))
    
    res.observed.plot(ax=axes[0], legend=False)
    axes[0].set_ylabel('Observado')
    
    res.trend.plot(ax=axes[1], legend=False)
    axes[1].set_ylabel('Tendência')
    
    res.seasonal.plot(ax=axes[2], legend=False)
    axes[2].set_ylabel('Sazonalidade')
    
    axes[2].annotate('Fevereiro',fontsize=10 
            ,xytext=(dt.datetime(2010,5,1), -33)
            ,xy=(dt.datetime(2010,2,1), -31) 
            ,arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color='black'))

    axes[2].annotate('Dezembro', fontsize=10
            ,xytext=(dt.datetime(2011,3,1), 40)
            ,xy=(dt.datetime(2010,12,1), 41)
           ,arrowprops=dict(arrowstyle="->", connectionstyle="arc3", color='black'))
    
    res.resid.plot(ax=axes[3], legend=False)
    axes[3].set_ylabel('Resíduo')
    
    plt.xlabel('')
    
plot_decompose(res)

# Seleção dos melhores parâmetros com o "auto_arima" <a name="4"></a>
Dando continuidade às previsões, iremos selecionar o modelo automaticamente através da função "auto_arima", que irá nos entregar os melhores componentes para o modelo ARIMA e suas estatísticas.

Em geral, as duas estatísticas levadas em consideração para a seleção do modelo, de tal forma que quanto menor o valor, melhor, são:

- AIC: Akaike Information Criteria
- BIC: Bayesian Information Criteria

In [ ]:
%%time
step_wise=auto_arima(acidentes_cvit,
                     start_p=0, start_q=0, 
                     max_p=3, max_q=3,
                     start_P=0, start_Q=0, 
                     max_P=3, max_Q=3,
                     d=1, max_d=1,
                     D=1, max_D=1,
                     m=12,
                     trace=True, 
                     error_action='ignore', 
                     suppress_warnings=True, 
                     stepwise=True)

In [ ]:
step_wise.summary()

# Ajustando o modelo SARIMA <a name="5"></a>
Após as diversas iterações que duraram aproximadamente 6 minutos, o modelo indicado foi o "SARIMAX(0, 1, 1)x(0, 1, 1, 12)". Observe que o "X" em SARIMA"X" exite pois é possível utilizar uma séria "exógena", ou externa, mas não será o nosso caso. Iremos aplicar o modelo SARIMA (Seasonal Autoregressive Integrated Moving Average).

In [ ]:
model = sm.tsa.statespace.SARIMAX(acidentes_cvit
                                 ,order=(0, 1, 1)
                                 ,seasonal_order=(0, 1, 1, 12)
                                 ,enforce_stationarity=False
                                 ,enforce_invertibility=False
                                 )
results = model.fit()
print(results.summary()) 

Aqui nossa principal preocupação deve ser de garantir que os resíduos do modelo não são correlacionados e igualmente distribuídos. Se isto não for alcançado, é indicativo de que os parâmetros podem ser melhorados.

No gráfico do canto superior direito, quando a linha vermelha do KDE (Kernel Density Estimation) está próxima da linha verde N(0,1), que significa "Distribuição Normal com média 0 e desvio padrão 1", é um bom indicativo de que os resíduos são igualmente distribuídos.

No canto inferior esquerdo, o gráfico mostra que a distribuição ordenada dos resíduos seguem a linha de tendência das amostras tiradas de uma distribuição normal com N(0, 1), com alguns desvios.

No gráfico de resíduos ao longo do tempo, no canto superior esquerdo, não há nenhuma sazonalidade aparente e parece se tratar de "ruído branco". O conceito de ruído branco significa que os dados são aleatórios e não podem ser preditos, pois não seguem um padrão. Isto é confirmado pelo "correlograma", no canto inferior direito, que mostra que os resíduos possuem baixa correrelação com os próprios lags.

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

# Validando as previsões <a name="6"></a>
Já ajustamos o modelo e agora podemos utilizá-lo para realizar previsões. Iremos começar comparando os valores preditos com os valores reais para nos auxiliar a entender sua precisão.

Iremos utilizar **"get_predition"** a partir de uma data **"X"** e com um intervalo de confiança **"conf_int"**.

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2015-01-01'), dynamic=False)
pred_ci = pred.conf_int()

fig, ax = plt.subplots(figsize=(20, 8))

acidentes_cvit.plot(ax=ax)
pred.predicted_mean.plot(ax=ax, alpha=0.8, color='r')

ax.legend(['Histórico', 'Previsão'], fontsize=15)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=0.07)
ax.set_xlabel('')
ax.set_ylabel('Preço')
plt.xticks(fontsize=15)
plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = acidentes_cvit.loc['2015-01-01':, 'com_vitimas']

# Compute the mean square error
mse = ((y_forecasted - y_truth) ** 2).mean()
print('O Erro Quadrático Médio é {}'.format(round(mse, 2)))

# Realizando e Visualizando Previsões <a name="7"></a>
Chegamos ao ponto principal da nossa análise, realizar as previões. Para isso iremos utilizar o atributo **"get_forecast"** que consegue computar os valores previstos com **N** passos a frente.

In [ ]:
pred_uc = results.get_forecast(steps=12)
pred_ci = pred_uc.conf_int()

fig, ax = plt.subplots(figsize=(20, 8))

ax = acidentes_cvit.plot(ax=ax)
pred_uc.predicted_mean.plot(ax=ax, color='g')

ax.legend(['Histórico', 'Previsão'], fontsize=15)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.06)
ax.set_xlabel('')
ax.set_ylabel('Preço')
plt.xticks(fontsize=15)
plt.show()

In [ ]:
# Intervalo de Confiança e valor médio
pred_ci['Previsão Média'] = (pred_ci.iloc[:, 0] +  pred_ci.iloc[:, 1]) / 2
pred_ci

# Previsão 2020
Após realizados todos os passos e encontrados os valores inferiores e superiores do intervalo de confiança, podemos dizer que previsão do número total de acidentes com vítimas para o ano de 2020 será de, aproximadamente:

>**6.940** acidentes com vítimas.

Seguindo a tendência de crescimento que fora identificada nos gráficos da análise exploratória.

In [ ]:
round(pred_ci['Previsão Média'].sum())

# Visualização Completa
Abaixo iremos visualizar a ilustração gráfica de todo histórico conhecido mais a previzão realizada pelo modelo SARIMAX, em verde.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))

preds = pd.DataFrame(pd.concat([acidentes_cvit, pred_ci['Previsão Média']]).sum(axis=1)).rename(columns={0:'com_vitimas'})
plt.plot(preds, linewidth=2, zorder=1)
plt.plot(preds.iloc[-12:,0], linewidth=3, color='y')
ax.legend(['Histórico', 'Previsão'], fontsize=15)
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)

plt.axvline(x="2020-01-01", ymin=0, ymax=700, ls='--')
plt.show()